# 10.1处理登录表单

## 10.1.1	处理登录表单

In [6]:
import requests
session = requests.session()

post_url = 'http://www.santostang.com/wp-login.php'
agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
headers = {
    "Host": "www.santostang.com",
    "Origin":"http://www.santostang.com",
    "Referer":"http://www.santostang.com/wp-login.php",
    'User-Agent': agent
}
postdata = {
    'pwd': 'a12345',
    'log': 'test',
    'rememberme' : 'forever',
    'redirect_to': 'http://www.santostang.com/wp-admin/',
    'testcookie' : 1,
}

login_page = session.post(post_url, data=postdata, headers=headers)
print(login_page.status_code)

200


In [12]:
import requests
import http.cookiejar as cookielib

session = requests.session()
session.cookies = cookielib.LWPCookieJar(filename='cookies')

post_url = 'http://www.santostang.com/wp-login.php'
agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
headers = {
    "Host": "www.santostang.com",
    "Origin":"http://www.santostang.com",
    "Referer":"http://www.santostang.com/wp-login.php",
    'User-Agent': agent
}
postdata = {
    'pwd': 'a12345',
    'log': 'test',
    'rememberme' : 'forever',
    'redirect_to': 'http://www.santostang.com/wp-admin/',
    'testcookie' : 1,
}

login_page = session.post(post_url, data=postdata, headers=headers)
print(login_page.status_code)
session.cookies.save()

200


## 10.1.2	处理cookies，让网页记得你的登录

In [13]:
import requests
import http.cookiejar as cookielib

In [14]:
session = requests.session()
session.cookies = cookielib.LWPCookieJar(filename='cookies')
try:
    session.cookies.load(ignore_discard=True)
except:
    print("Cookie 未能加载")

In [15]:
def isLogin():
    url = "http://www.santostang.com/wp-admin/profile.php"
    login_code = session.get(url, headers=headers, allow_redirects=False).status_code
    if login_code == 200:
        return True
    else:
        return False

In [16]:
if __name__ == '__main__':
    agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
    headers = {
        "Host": "www.santostang.com",
        "Origin":"http://www.santostang.com",
        "Referer":"http://www.santostang.com/wp-login.php",
        'User-Agent': agent
    }
    if isLogin():
        print('您已经登录')

您已经登录


## 10.1.3	完整的登录代码

In [18]:
import requests
import http.cookiejar as cookielib

session = requests.session()
session.cookies = cookielib.LWPCookieJar(filename='cookies')
try:
    session.cookies.load(ignore_discard=True)
except:
    print("Cookie 未能加载")

def isLogin():
    # 通过查看用户个人信息来判断是否已经登录
    url = "http://www.santostang.com/wp-admin/profile.php"
    login_code = session.get(url, headers=headers, allow_redirects=False).status_code
    if login_code == 200:
        return True
    else:
        return False
    
def login(secret, account):
    post_url = 'http://www.santostang.com/wp-login.php'
    postdata = {
        'pwd': secret,
        'log': account,
        'rememberme' : 'true',
        'redirect_to': 'http://www.santostang.com/wp-admin/',
        'testcookie' : 1,
    }
    try:
        # 不需要验证码直接登录成功
        login_page = session.post(post_url, data=postdata, headers=headers)
        login_code = login_page.text
        print(login_page.status_code)
        #print(login_code)
    except:
        pass
    session.cookies.save()
    
if __name__ == '__main__':
    agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
    headers = {
        "Host": "www.santostang.com",
        "Origin":"http://www.santostang.com",
        "Referer":"http://www.santostang.com/wp-login.php",
        'User-Agent': agent
    }
    if isLogin():
        print('您已经登录') 
    else:
        login('a12345', 'test') 

Cookie 未能加载
200


# 10.2验证码的处理

## 10.2.2	人工方法处理验证码

In [12]:
def get_captcha():
    #获取验证码图片所在的url
    r = session.get('http://www.santostang.com/wp-login.php?action=register', headers=headers)
    soup = BeautifulSoup(r.text, "lxml")
    captcha_url = soup.find("img", id="captcha_code_img")["src"]
    # 获取验证码图片
    r = session.get(captcha_url, headers=headers)
    with open('captcha.jpg', 'wb') as f:
        f.write(r.content)
        f.close()
    try:
        im = Image.open('captcha.jpg')
        im.show()
        im.close()
    except:
        print(u'请到 %s 目录找到captcha.jpg 手动输入' % os.path.abspath('captcha.jpg'))
    captcha = input("please input the captcha\n>")
    return captcha

In [13]:
def register(account, email):      
    post_url = 'http://www.santostang.com/wp-login.php?action=register'
    postdata = {
        'user_login': account,
        'user_email': email,
        'redirect_to': '',
        }
    # 调用get_captcha函数，获取验证码数字
    postdata["ux_txt_captcha_challenge_field"] = get_captcha()  
    # 提交POST请求，进行注册
    register_page = session.post(post_url, data=postdata, headers=headers)
    # 若输出打印结果为200，则表示注册成功
    print(register_page.status_code)

In [14]:
import requests
from bs4 import BeautifulSoup
import re
import os
from PIL import Image
if __name__ == '__main__':
    agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
    headers = {
        "Host": "www.santostang.com",
        "Origin":"http://www.santostang.com",
        "Referer":"http://www.santostang.com/wp-login.php",
        'User-Agent': agent
    }
    session = requests.session()
    # 调用注册函数进行注册
    account = '18341432113' #改成自己用户名
    email = 'a12345@qq.com' # 改成自己邮箱
    register(account, email)

please input the captcha
>3p5e
200


## 10.2.3	OCR处理验证码

In [4]:
from PIL import Image
im = Image.open('captcha.jpg')
gray = im.convert('L')
gray.show()
gray.save("captcha_gray.jpg")

In [5]:
threshold = 150
table = []
for i in range(256):
    if i < threshold:
        table.append(0)
    else:
        table.append(1)
out = gray.point(table, '1')
out.show()
out.save("captcha_thresholded.jpg")

In [2]:
import pytesseract
th = Image.open('captcha_thresholded.jpg')
print(pytesseract.image_to_string(th))

WD6N
